### Convolutional Neural Networks

#### torch.nn 

* conv2d
* Relu
* MaxPool2d
* pytorch cnn (basic pytorch tutorial)
* cifar10 and cifar100
* imagenet (1000 class)

### Use Pretrained Model

* AlexNet
* VGGNet
* ResNet


In [2]:
import torch

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets
import torchvision.models as models

# Load models
resnet18 = models.resnet18(pretrained=True)
alexnet = models.alexnet(pretrained=True)
vgg16 = models.vgg16(pretrained=True)
squeezenet = models.squeezenet1_0(pretrained=True)
densenet = models.densenet161(pretrained=True)
inception = models.inception_v3(pretrained=True)
googlenet = models.googlenet(pretrained=True)
shufflenet = models.shufflenet_v2_x1_0(pretrained=True)
mobilenet = models.mobilenet_v2(pretrained=True)

models_list = [
    ("resnet18", resnet18),
    ("alexnet", alexnet),
    ("vgg16", vgg16),
    ("squeezenet", squeezenet),
    ("densenet", densenet),
    ("inception_v3", inception),
    ("googlenet", googlenet),
    ("shufflenet", shufflenet),
    ("mobilenet_v2", mobilenet),
]

# Prepare the MNIST dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 for pre-trained models
    transforms.Grayscale(3),       # Convert 1-channel grayscale to 3-channel
    transforms.ToTensor(),         # Convert images to tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

test_dataset = datasets.MNIST(root="./data", train=False, transform=transform, download=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

device = torch.device("cpu")

for model_name, model in models_list:
    print(f"\nLoading model: {model_name}")

    # Modify the model for MNIST
    if isinstance(model, models.Inception3):
        model.aux_logits = False  # Disable auxiliary outputs for Inception v3

    if hasattr(model, "fc"):
        in_features = model.fc.in_features
        model.fc = nn.Linear(in_features, 10)
    elif hasattr(model, "classifier"):
        if isinstance(model.classifier, nn.Sequential):
            in_features = model.classifier[-1].in_features
            model.classifier[-1] = nn.Linear(in_features, 10)
        else:
            in_features = model.classifier.in_features
            model.classifier = nn.Linear(in_features, 10)

    model = model.to(device)
    model.eval()

    print("Running in prediction mode...")
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Accuracy of {model_name} on MNIST test data: {accuracy:.2f}%")
